In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torch.nn as nn

In [2]:
building = pd.read_csv("data/outer.csv")
building.head()

,관리_건축물대장_PK,관리_상위_건축물대장_PK,새주소_지상지하_코드,주소,지상층수,지하층수,승강기_수,주건축물수,세대수,가구수,동명칭,호_명,층_구분_코드_x,층_구분_코드_y,층_번호,면적,용적율,주차_대수
0,11530-602,NaN,지상,서울특별시 구로구 경인로53길 15,NaN,NaN,NaN,5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,174.92,3421.0
1,11590-100241986,11590-100241435,지상,서울특별시 동작구 사당로23길 112,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
2,11530-7503,NaN,지상,서울특별시 구로구 구로중앙로 217,8.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.18,49.0
3,11530-2719,11530-844,지상,서울특별시 구로구 오리로21가길 47-10,3.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0
4,11530-2718,11530-844,지상,서울특별시 구로구 오리로21가길 47-10,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.0


In [3]:
#GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [5]:
building.fillna(0.0, inplace = True)
building["업종"] = 0

In [6]:
X = building.drop(["업종", "주소", "관리_건축물대장_PK", "관리_상위_건축물대장_PK", "새주소_지상지하_코드", "층_구분_코드_x", "동명칭", "호_명"], axis = 1).to_numpy()
y = building["업종"].to_numpy()

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_test = torch.from_numpy(y_test).to(device)

In [9]:
X_train[1].shape

torch.Size([11])

In [10]:
class MLP(torch.nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(11, 11),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(11, 11),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(11, 14),
        )
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(list(zip(X_train, y_train)), batch_size = 64,  shuffle = True)
test_dataloader = DataLoader(list(zip(X_test, y_test)), batch_size = 64, shuffle = True)

In [14]:
lr = 0.001
epochs = 20
model = MLP().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = lr)

In [15]:
## train
for epoch in range(epochs):
    for batch_idx, (X_train, y_train) in enumerate(train_dataloader):
        model.train()
        optimizer.zero_grad()
        X_train = X_train.to(device, dtype=torch.float)
        y_train = y_train.to(device, dtype=torch.long)
        pred = model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            loss, current = loss.item(), batch_idx * len(X_train)
            print(f"loss: {loss:>7f} [{current:>5d}/{len(train_dataloader.dataset):>5d}]")
        
    test_loss, test_correct, val_loss, val_correct = 0, 0, 0, 0
    model.eval()

    with torch.no_grad():
        for X_test, y_test in test_dataloader:
            X_test = X_test.to(device, dtype=torch.float)
            y_test = y_test.to(device, dtype=torch.long)
            pred = model(X_test)
            test_loss += criterion(pred, y_test).item()
            test_correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()
        test_loss /= len(test_dataloader)
        test_correct /= len(test_dataloader.dataset)
        print(f"test Error: \nAccuracy: {(100*test_correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000

loss: 0.000000 [    0/22375]
loss: 0.000000 [ 6400/22375]
loss: 0.000000 [12800/22375]
loss: 0.000000 [19200/22375]
test Error: 
Accuracy: 100.0%, Avg loss: 0.000000